In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:

# Input
# Input = [[ 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]] -> 1

# Adjusted Input: Expects shape (10,) for an array of 10 integers
tf.random.set_seed(42)
input_layer = tf.keras.layers.Input(shape=(10, 1))
output_layer = tf.keras.layers.Dense(units=10 , activation = 'linear')# units=1 for average
sequential_model = tf.keras.Sequential(
    [   
        tf.keras.layers.Dense(units=75 , activation = 'selu'),
        tf.keras.layers.Conv1D(filters=128, strides=2 , kernel_size=10, activation='selu' , kernel_regularizer=tf.keras.regularizers.L2(0.003)),
        tf.keras.layers.Dense(units=64 , activation = 'selu' , kernel_regularizer=tf.keras.regularizers.L2(0.003)),
        tf.keras.layers.Conv1D(filters=64, strides=2 , kernel_regularizer=tf.keras.regularizers.L2(0.003),kernel_size=5, activation='selu', padding='same'),
        tf.keras.layers.LSTM(units=64, kernel_regularizer=tf.keras.regularizers.L2(0.003) , return_sequences=True),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.PReLU(),
        tf.keras.layers.LSTM(units=50 , kernel_regularizer=tf.keras.regularizers.L2(0.003)),
        tf.keras.layers.Dropout(rate=0.2),

        
    ]
)

pass_output= sequential_model(input_layer)
output = output_layer(pass_output)

model = tf.keras.models.Model(inputs = input_layer , outputs = output)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , optimizer=tf.keras.optimizers.Adam(10e-7) , metrics=['accuracy'])
# model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , optimizer=tf.keras.optimizers.Adam(1e-5) , metrics=['accuracy'])
#loss='mse'


#keras tuner daganna

In [ ]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 50)             │       181,166 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 181,676 (709.67 KB)

 Trainable params: 181,676 (709.67 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Test train - > 60% 30% 10%
from sklearn.model_selection import train_test_split

X = pd.read_csv("previous_values_2.csv")
Y = pd.read_csv("y_values_2.csv")

# X = pd.read_csv("probability_of_last10numbers.csv")
# Y = pd.read_csv("next_number.csv")

X

,Prev_1,Prev_2,Prev_3,Prev_4,Prev_5,Prev_6,Prev_7,Prev_8,Prev_9,Prev_10
0,7.0,5.0,5.0,0.0,8.0,6.0,8.0,2.0,6.0,2.0
1,7.0,7.0,5.0,5.0,0.0,8.0,6.0,8.0,2.0,6.0
2,9.0,7.0,7.0,5.0,5.0,0.0,8.0,6.0,8.0,2.0
3,5.0,9.0,7.0,7.0,5.0,5.0,0.0,8.0,6.0,8.0
4,9.0,5.0,9.0,7.0,7.0,5.0,5.0,0.0,8.0,6.0
...,...,...,...,...,...,...,...,...,...,...
4999985,2.0,1.0,6.0,1.0,3.0,9.0,3.0,4.0,9.0,5.0
4999986,5.0,2.0,1.0,6.0,1.0,3.0,9.0,3.0,4.0,9.0
4999987,4.0,5.0,2.0,1.0,6.0,1.0,3.0,9.0,3.0,4.0
4999988,4.0,4.0,5.0,2.0,1.0,6.0,1.0,3.0,9.0,3.0


In [ ]:
# X_train_orig, X_test, Y_train_orig, Y_test = train_test_split(X, Y, test_size=0.1,stratify=Y) 
# X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_orig,Y_train_orig, test_size=0.3,stratify=Y_train_orig) 

test_size = int(len(X) * 0.1)  # Calculate the size of the test set (10% of the data)

X_test = X[:test_size]  # Take the first 10% of the data for X_test
X_train_orig = X[test_size:]  # Take the remaining 90% of the data for X_train
Y_test = Y[:test_size]  # Take the corresponding labels for the test set
Y_train_orig = Y[test_size:]  # Take the corresponding labels for the training set

X_valid = X_train_orig[:test_size]  # Take the first 10% of the data for X_test
X_train = X_train_orig[test_size:]  # Take the remaining 90% of the data for X_train
Y_valid = Y_train_orig[:test_size]  # Take the corresponding labels for the test set
Y_train = Y_train_orig[test_size:]  # Take the corresponding labels for the training set

In [ ]:
X_test

,Prev_1,Prev_2,Prev_3,Prev_4,Prev_5,Prev_6,Prev_7,Prev_8,Prev_9,Prev_10
0,7.0,5.0,5.0,0.0,8.0,6.0,8.0,2.0,6.0,2.0
1,7.0,7.0,5.0,5.0,0.0,8.0,6.0,8.0,2.0,6.0
2,9.0,7.0,7.0,5.0,5.0,0.0,8.0,6.0,8.0,2.0
3,5.0,9.0,7.0,7.0,5.0,5.0,0.0,8.0,6.0,8.0
4,9.0,5.0,9.0,7.0,7.0,5.0,5.0,0.0,8.0,6.0
...,...,...,...,...,...,...,...,...,...,...
499994,0.0,9.0,6.0,6.0,6.0,2.0,2.0,4.0,7.0,7.0
499995,9.0,0.0,9.0,6.0,6.0,6.0,2.0,2.0,4.0,7.0
499996,1.0,9.0,0.0,9.0,6.0,6.0,6.0,2.0,2.0,4.0
499997,3.0,1.0,9.0,0.0,9.0,6.0,6.0,6.0,2.0,2.0


In [ ]:
Y_test

,Generated Numbers
0,7
1,9
2,5
3,9
4,0
...,...
499994,9
499995,1
499996,3
499997,9


In [ ]:
X_train

,Prev_1,Prev_2,Prev_3,Prev_4,Prev_5,Prev_6,Prev_7,Prev_8,Prev_9,Prev_10
999998,4.0,6.0,3.0,7.0,7.0,0.0,4.0,2.0,0.0,8.0
999999,6.0,4.0,6.0,3.0,7.0,7.0,0.0,4.0,2.0,0.0
1000000,3.0,6.0,4.0,6.0,3.0,7.0,7.0,0.0,4.0,2.0
1000001,9.0,3.0,6.0,4.0,6.0,3.0,7.0,7.0,0.0,4.0
1000002,1.0,9.0,3.0,6.0,4.0,6.0,3.0,7.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4999985,2.0,1.0,6.0,1.0,3.0,9.0,3.0,4.0,9.0,5.0
4999986,5.0,2.0,1.0,6.0,1.0,3.0,9.0,3.0,4.0,9.0
4999987,4.0,5.0,2.0,1.0,6.0,1.0,3.0,9.0,3.0,4.0
4999988,4.0,4.0,5.0,2.0,1.0,6.0,1.0,3.0,9.0,3.0


In [ ]:
Y_train

,Generated Numbers
999998,6
999999,3
1000000,9
1000001,1
1000002,8
...,...
4999985,5
4999986,4
4999987,4
4999988,2


In [ ]:
X_valid

,Prev_1,Prev_2,Prev_3,Prev_4,Prev_5,Prev_6,Prev_7,Prev_8,Prev_9,Prev_10
499999,4.0,9.0,3.0,1.0,9.0,0.0,9.0,6.0,6.0,6.0
500000,6.0,4.0,9.0,3.0,1.0,9.0,0.0,9.0,6.0,6.0
500001,9.0,6.0,4.0,9.0,3.0,1.0,9.0,0.0,9.0,6.0
500002,0.0,9.0,6.0,4.0,9.0,3.0,1.0,9.0,0.0,9.0
500003,5.0,0.0,9.0,6.0,4.0,9.0,3.0,1.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...
999993,0.0,4.0,2.0,0.0,8.0,0.0,7.0,0.0,1.0,3.0
999994,7.0,0.0,4.0,2.0,0.0,8.0,0.0,7.0,0.0,1.0
999995,7.0,7.0,0.0,4.0,2.0,0.0,8.0,0.0,7.0,0.0
999996,3.0,7.0,7.0,0.0,4.0,2.0,0.0,8.0,0.0,7.0


In [ ]:
Y_valid

,Generated Numbers
499999,6
500000,9
500001,0
500002,5
500003,4
...,...
999993,7
999994,7
999995,3
999996,6


In [ ]:
Y.dtypes

Generated Numbers    int64
dtype: object

In [ ]:
X.shape,Y.shape

((4999990, 10), (4999990, 1))

In [ ]:
X_train.shape,X_valid.shape,X_test.shape

((3999992, 10), (499999, 10), (499999, 10))

In [ ]:
Y_train.shape,Y_valid.shape,Y_test.shape

((3999992, 1), (499999, 1), (499999, 1))

In [ ]:
# batch_val = [10, 50, 100, 200, 500, 1000]

# for batch in batch_val:
#     pass

In [ ]:
Y_train

,Generated Numbers
999998,6
999999,3
1000000,9
1000001,1
1000002,8
...,...
4999985,5
4999986,4
4999987,4
4999988,2


In [ ]:
history = model.fit(X_train, Y_train, 
          batch_size=3000, 
          verbose=1, 
          epochs=1, 
          validation_data=(X_valid, Y_valid),
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss",mode="min" ,patience=3)])

1334/1334 ━━━━━━━━━━━━━━━━━━━━ 52s 36ms/step - accuracy: 0.1000 - loss: 3.6173 - val_accuracy: 0.0997 - val_loss: 3.5800


In [ ]:
model.save('my_model.keras')

In [ ]:
# loading model

model_test = tf.keras.models.load_model('my_model.keras')

In [ ]:
model_test.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 50)             │       181,166 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 545,030 (2.08 MB)

 Trainable params: 181,676 (709.67 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 363,354 (1.39 MB)

In [ ]:
model.predict(X_test)

15625/15625 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step


array([[-0.0046743 ,  0.0061735 , -0.00279553, ...,  0.00088544,
        -0.00100522,  0.00239474],
       [-0.00761043, -0.00582764, -0.02326608, ..., -0.0039332 ,
         0.00797183,  0.00522177],
       [-0.00455144, -0.0012344 , -0.01568533, ..., -0.00464002,
         0.00237013,  0.00177117],
       ...,
       [-0.00698152, -0.00464762, -0.01577568, ...,  0.0058489 ,
         0.01085654,  0.01032358],
       [-0.00568866, -0.00131803,  0.00019089, ...,  0.00977771,
         0.004078  , -0.00266148],
       [-0.00555782, -0.00028522, -0.01781921, ..., -0.00190755,
         0.00430828,  0.00475299]], dtype=float32)

In [ ]:
values  = tf.nn.softmax(model.predict(X_test))#not for average

15625/15625 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step


In [ ]:
predicted_classes = np.argmax(values , axis = 1)#not for average

In [ ]:
print(predicted_classes)#not for average
print(Y_test['Generated Numbers'])


[1 8 8 ... 8 7 5]
0         7
1         9
2         5
3         9
4         0
         ..
499994    9
499995    1
499996    3
499997    9
499998    4
Name: Generated Numbers, Length: 499999, dtype: int64


In [ ]:
# import random


# def sum_to_index(arr, i):
#   sum = 0
#   for j in range(i + 1):  # Include the element at the i-th index
#     sum += arr[j]
#   return sum

# real_trade_starting_point = 40   # margin of real trade starts
# possible_real_points = 40        # 25 for 95 dollar capital (max 51 for 2105 dollars capital)
# stopping_profit = 30            # trade stop point while another outlier comes

# i = 0
# c = 0
# p = 0
# Tp = 0
# ml = [1,1,1,1,1,1,1,2,2,2,2,2,3,3,3,4,4,5,5,6,7,8,9,10,11,12,14,16,18,20,22,25,28,32,36,40,45,50,57,64,72,81,91,103,116,130,146,165,185,208,234]
# RT1 = False
# RT2 = False
# Rc = 0

# digit_array = []

# for n in predicted_classes :
#     print(type(n))
#     if not(RT2) :
#         if (n == Y_test['Generated Numbers'][i]) :
#             if c >= real_trade_starting_point + possible_real_points :
#                 RT1 = True
#             c=0
#         else  :
#             c += 1         
#     elif RT2 :
#         if (n == Y_test['Generated Numbers'][i]) and Rc < possible_real_points - 1 :
#             p += ml[Rc+1]*8 - sum_to_index(ml, Rc)
#             Tp += ml[Rc+1]*8 - sum_to_index(ml, Rc)
#             with open("REAL_wins.csv", 'a', newline='') as f:  
#                 f.write(f"consequent : {Rc} , Profit : {Tp}\n")
#             Rc=0

#             if p // stopping_profit == 1 :
#                 RT1 = False
#                 RT2 = False
#                 p = 0
#                 Rc=0
#                 c = 0
#         else  :
#             Rc += 1
#             if Rc == possible_real_points :
#                 RT1 = False
#                 RT2 = False
#                 p = 0
#                 Rc=0
#                 c = 0
#                 Tp = Tp - sum_to_index(ml,(possible_real_points - 1))
#                 with open("REAL_wins.csv", 'a', newline='') as f:  
#                     f.write(f"!!!!!!!!!!!!!!!!!!!!!!!!!!! Profit : {Tp} !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")

#     if RT1 and c == real_trade_starting_point :
#         RT2 = True

#     i += 1

# print(len(Y_test))

# # while True :

# #     for k in range(11):
# #         if k <=9:
# #             digit_array.append(random.randint(0,9))
# #         elif k ==10:
# #             recieved_number = random.randint(0,9)
    
    

# #     array_1d = np.array(digit_array)
# #     array_2d = array_1d.reshape(1, 10)

# #     df = pd.DataFrame(array_2d)
# #     pre = model_test.predict(df)
# #     predictions = np.argmax(pre , axis = 1)
# #     print(predictions[0])

# #     if not(RT2) :
# #         if ( predictions[0] == recieved_number) :
# #             if c >= real_trade_starting_point + possible_real_points :
# #                 RT1 = True
# #             c=0
# #         else  :
# #             c += 1         
# #     elif RT2 :
# #         if (predictions[0] == recieved_number) and Rc < possible_real_points - 1 :
# #             p += ml[Rc+1]*8 - sum_to_index(ml, Rc)
# #             Tp += ml[Rc+1]*8 - sum_to_index(ml, Rc)
# #             with open("REAL_wins.csv", 'a', newline='') as f:  
# #                 f.write(f"consequent : {Rc} , Profit : {Tp}\n")
# #             Rc=0

# #             if p // stopping_profit == 1 :
# #                 RT1 = False
# #                 RT2 = False
# #                 p = 0
# #                 Rc=0
# #                 c = 0
# #         else  :
# #             Rc += 1
# #             if Rc == possible_real_points :
# #                 RT1 = False
# #                 RT2 = False
# #                 p = 0
# #                 Rc=0
# #                 c = 0
# #                 Tp = Tp - sum_to_index(ml,(possible_real_points - 1))
# #                 with open("REAL_wins.csv", 'a', newline='') as f:  
# #                     f.write(f"!!!!!!!!!!!!!!!!!!!!!!!!!!! Profit : {Tp} !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")

# #     if RT1 and c == real_trade_starting_point :
# #         RT2 = True
    
# #     digit_array = []

#     # print('recieved_number : ',recieved_number,'     predictions : ',predictions[0],'   profit : ',Tp)



In [ ]:
from main_file import run
from save_digits import get_last_digits
import asyncio

async def main():
    # Run function_1 and function_2 concurrently
    await asyncio.gather(run(), get_last_digits())

if __name__ == "__main__":

    asyncio.run(await main())

In [ ]:
pr = (predicted_classes > 5).astype(int)#not for average
pr#not for average

In [ ]:
len(pr)

In [ ]:
act = (Y_test>5).astype(int)
act

In [ ]:
pr_series = pd.Series(pr)
# asd = (act['Generated Numbers'] == pr_series ).astype(int)
# Ensure both Series have the same index (if not already)

asd1 = ((act['Generated Numbers'] == 1) & (pr_series == 1)).astype(int)
asd0 = ((act['Generated Numbers'] == 1) & (pr_series == 0)).astype(int)



In [ ]:
i = 0
w = 0
l = 0
c = 0
mc =0
for n in pr_series :
    if (n == 1 and act['Generated Numbers'][i] == 1 ) :
        w +=1
        if mc < c :
            mc = c
        with open("wins.csv", 'a', newline='') as f:  
            f.write(f"consequent : {c}\n")
        c=0
    elif (n == 1 and act['Generated Numbers'][i] == 0 ) :
        l += 1
        c += 1
    i += 1

print('wins  : ',w)
print('losts : ',l)
print('max consequent :',mc)
print('profit: ',w*1.44 -l)

In [ ]:
# Assuming your trained model is stored in 'model'
# Assuming the history object is available (captured during training)

# Extract training and validation loss from history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the loss curves
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
predictions = predicted_classes#not for average. predictions = model.predict(X_test)

# Extract the last 100 predicted and actual values (assuming enough test data)
last_100_predictions = predictions[-100:]
last_100_actuals = Y_test[-100:]

# Flatten predictions
last_100_predictions_flat = np.ravel(last_100_predictions)

# Create the plot
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.plot(range(len(last_100_actuals)), last_100_actuals, label='Actual Values')
plt.plot(range(len(last_100_predictions_flat)), last_100_predictions_flat, label='Predicted Values')
plt.xlabel('Sample Index (Last 100)')
plt.ylabel('Values')
plt.title('Actual vs. Predicted Values (Last 100)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
# Assuming one-hot encoded labels
accuracy = accuracy_score(Y_test, predicted_classes)
print("Accuracy:", accuracy)

#precision